In [1]:
import pandas as pd
from IPython.display import display
import mysql.connector

In [2]:
# Functions that used to connect with the cloud database and display results.
def create_connection():
    conn = None
    try:
       conn = mysql.connector.connect(
          user="DB00004175",
          password="tTNlXEirHh)EmwXhtp4koTR",
          host="menu-data-db.mdb0002067.db.skysql.net",
          ssl_ca="skysql_chain.pem",
          port=5001)
       conn.autocommit = False
       return conn
    except mysql.connector.Error as e:
       print(f"Error connecting to MariaDB Platform: {e}")
    return conn

def sql_execute(conn, sql):

    try:
        cur = conn.cursor()
        cur.execute(sql, multi=True)
        conn.commit()
    except mysql.connector.Error as e:
        print(e)

def sql_execute_show(conn, sql):

    try:
        df = pd.read_sql(sql, con = conn)
        #display(df)
        return df
    except mysql.connector.Error as e:
        print(e)

In [3]:
conn = create_connection()


In [4]:
sql_execute_show(conn, sql = '''
SELECT * FROM menu_cstore.nutrition_facts;
''')

,item_id,year,r_id,cat_id,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,potassium,carbs,protein,sugar,dietary_Fiber
0,7,2008,10.0,3,290.0,12.0,45.0,5.0,35.0,560.0,NaN,30.0,15.0,6.0,1.0
1,7,2010,10.0,3,260.0,11.0,4.0,0.0,30.0,520.0,NaN,27.0,14.0,5.0,1.0
2,7,2012,10.0,3,260.0,10.0,4.0,0.0,35.0,490.0,NaN,28.0,13.0,6.0,1.0
3,7,2013,10.0,3,240.0,8.0,4.0,0.0,30.0,460.0,NaN,31.0,12.0,7.0,1.0
4,7,2014,10.0,3,230.0,9.0,3.0,0.0,25.0,460.0,NaN,26.0,9.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209527,179836,2018,95.0,11,100.0,6.0,1.0,0.0,0.0,860.0,NaN,11.0,0.0,0.0,1.0
209528,179837,2018,95.0,11,140.0,9.0,1.0,0.0,20.0,710.0,NaN,16.0,1.0,8.0,4.0
209529,179838,2018,95.0,11,140.0,1.0,0.0,0.0,1.0,510.0,NaN,32.0,1.0,28.0,0.0
209530,179839,2018,95.0,11,110.0,0.0,0.0,0.0,1.0,1180.0,NaN,25.0,2.0,23.0,0.0


In [6]:
avg_restaurant_stats = sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(calories), AVG(trans_fat), AVG(cholesterol), AVG(carbs), AVG(sugar), AVG(dietary_Fiber)
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')
avg_restaurant_stats.head()

,r_name,AVG(calories),AVG(trans_fat),AVG(cholesterol),AVG(carbs),AVG(sugar),AVG(dietary_Fiber)
0,None,459.4577,3.0645,72.6958,44.0661,18.2593,2.3239
1,7 Eleven,289.9115,0.0419,33.8743,29.8953,6.4503,2.0330
2,Applebee's,535.8140,0.8290,67.2923,45.8912,19.2851,3.3980
3,Arby's,347.4370,2.6063,42.3261,38.9258,15.3426,1.7426
4,Auntie Anne's,275.5607,0.0371,13.3483,53.0438,33.4180,0.6921


Not all fats are created equal, as some foods such as olive oil, nuts, and fish contain good fats. However, trans fats are known to be artery clogging, and so are bad for people who are suffering from heart diseases. We are interested to know which restaurants should be avoided by people who have/are prone to heart problems. We want to view the restaurants who have the highest trans fat/fat ratio. 

We find that Long John Silver's and Arby's are the worst.

In [12]:
avg_restaurant_stats.sort_values(by = 'AVG(cholesterol)', ascending = False)

,r_name,AVG(trans_fat),AVG(cholesterol),AVG(sodium)
40,Hooters,0.6361,224.5413,1921.0583
33,Famous Dave's,0.7467,198.7875,2885.8822
41,IHOP,0.7024,190.4557,1042.0655
48,Joe's Crab Shack,0.0201,162.4783,1600.6796
77,Romano's Macaroni Grill,0.5513,161.6875,1286.6627
...,...,...,...,...
4,Auntie Anne's,0.0371,13.3483,330.4551
20,Chipotle,0.0000,11.3216,268.6051
53,Long John Silver's,1.6259,10.8705,291.2043
44,Jamba Juice,0.0000,5.3876,133.2113


In [15]:
fat_ratio = sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(trans_fat)/AVG(total_fat) AS fat_ratio
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')

fat_ratio.sort_values(by = 'fat_ratio', ascending = False)

,r_name,fat_ratio
53,Long John Silver's,0.274404
3,Arby's,0.163831
87,TGI Friday's,0.136636
0,None,0.125190
22,Church's Chicken,0.070406
...,...,...
67,Papa Murphy's,0.000435
20,Chipotle,0.000000
44,Jamba Juice,0.000000
47,Jimmy John's,NaN


Similarly, we would like to look at restaurants that should be avoided by those who have type 2 diabetes. We do this by examining restaurants that have the highest sugar/calories ratio. Interestingly, we find that two of the five worst restaurants are beverage stores: Jamba Juice and Starbucks.

In [17]:
sugar_ratio = sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(sugar)/AVG(calories) AS sugar_ratio
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')

sugar_ratio.sort_values(by = 'sugar_ratio', ascending = False)

,r_name,sugar_ratio
44,Jamba Juice,0.167131
53,Long John Silver's,0.151758
83,Starbucks,0.143843
49,KFC,0.129735
50,Krispy Kreme,0.128621
...,...,...
77,Romano's Macaroni Grill,0.011868
23,Ci Ci's Pizza,0.010579
40,Hooters,0.010562
47,Jimmy John's,NaN


Now we are interested in restaurants that, on average, offers food with high amounts of fiber. Fiber is known to have the opposite effects of trans fats and sugar. It lowers blood sugars, and lowers cholesterol levels. 

In [19]:
fiber_ratio = sql_execute_show(conn, sql = '''
SELECT b.r_name, AVG(dietary_Fiber)/AVG(calories) AS fiber_ratio
FROM menu_cstore.nutrition_facts a
LEFT JOIN menu_cstore.restaurant b ON a.r_id = b.r_id
GROUP BY b.r_name
;
''')

fiber_ratio.sort_values(by = 'fiber_ratio', ascending = False)

,r_name,fiber_ratio
20,Chipotle,0.017741
73,Qdoba,0.016631
78,Round Table Pizza,0.012817
44,Jamba Juice,0.012557
15,Carrabba's Italian Grill,0.011705
...,...,...
53,Long John Silver's,0.002985
81,Sheetz,0.002891
25,Dairy Queen,0.002771
4,Auntie Anne's,0.002512


Overall, it seems like Long John Silver's is the worst restaurant to eat at, especially for a person struggling with type 2 diabetes as well as heart problems. Their food doesn't seem to be nutritious (low fiber). In this analysis, we note that Long John Silver's is consistently the least nutritious for you.